# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
file = pd.read_csv('../output_data/weather_df.csv')

weather_df = file
del weather_df["Unnamed: 0"]
weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hobart,-42.8794,147.3294,68.00,45,75,16.11,AU,1612661779
1,Punta Arenas,-53.1500,-70.9167,51.80,66,0,17.27,CL,1612661595
2,George Town,5.4112,100.3354,80.60,83,20,2.30,MY,1612661728
3,Puerto Ayora,-0.7393,-90.3518,77.00,93,50,6.69,EC,1612661537
4,Carnarvon,-24.8667,113.6333,84.20,100,40,13.80,AU,1612661784
...,...,...,...,...,...,...,...,...,...
577,Royan,45.6285,-1.0281,42.80,87,0,8.05,FR,1612662604
578,Koslan,63.4564,48.8989,-22.88,83,65,10.87,RU,1612662605
579,Mezhdurechensk,53.6942,88.0603,21.02,95,37,3.96,RU,1612662445
580,Saryg-Sep,51.5000,95.6000,2.44,90,0,3.49,RU,1612662608


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lng"]]
humidity_weight = weather_df["Humidity"].astype(float)

In [30]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [5]:
narrow_weather_df = weather_df.loc[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70) & 
                                    (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0), :]
narrow_weather_df = narrow_weather_df.dropna(how='any')
narrow_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
125,Iquique,-20.2208,-70.1431,73.40,49,0,9.22,CL,1612661958
135,Yangjiang,21.8500,111.9667,78.01,45,0,3.49,CN,1612661972
152,Assaí,-23.3733,-50.8414,75.20,57,0,5.75,BR,1612661996
170,Vao,-22.6667,167.4833,78.03,80,0,5.46,NC,1612662023
174,Lázaro Cárdenas,17.9583,-102.2000,73.83,75,0,1.74,MX,1612662029
176,Formosa,-26.1775,-58.1781,71.60,78,0,5.75,AR,1612662032
249,Crateús,-5.1783,-40.6775,76.32,57,0,3.04,BR,1612662136
274,Puerto Baquerizo Moreno,-0.9000,-89.6000,76.73,85,0,7.78,EC,1612662138
278,Ban Phai,16.0600,102.7310,78.01,50,0,7.40,TH,1612662177
283,Te Anau,-45.4167,167.7167,75.90,38,0,5.68,NZ,1612662184


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = narrow_weather_df

# Add Hotel Name column to hotel_df        
hotel_df["Hotel Name"] = ""

for index, row in hotel_df.iterrows():

    #base url
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #Setting parameters with dictionary to search for hotels
    params = {"location": str(row["Lat"]) + "," +str(row["Lng"]),
              "radius": str(5000),
              "type": "Hotel",
              "keyword": "Hotel",
              "key": g_key}
    
    response = requests.get(url, params=params).json()
    #Make sure it works by printing out the result
    print(response)    
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        

    except (KeyError, IndexError):
        print("Missing")    


{'html_attributions': [], 'next_page_token': 'ATtYBwJHd--ajT6J4nfuLbbJkeBPldYN6KqprC2eMIxuFP09QIekDFpI4fzCgGyHw6Q4GApaqUdMW6k-xoHGnoIwGZyHbNIJMRMpmYmjQdCgltMpTvp6TTkaVXxrd9zflZYf9HfQGZ4peTc4aTgYxCjJKbEdGAddybAi7vwYGgDk9Gv7Lqz0nKeu8WA_zMZr5aslvgux5CuyjZT1W1OZ5xUcYztvv7sIDpzcave0A5GWyfVZsmXIvFMy62Vp7mpVpBz6tB431axkQ3vUyceDpt0GFeHjIiLOdzVNEzcYAAqsj85ekgeTszWd3tu_rp-7DVTvKeFta0uH9vH3noEwEQBMKkILXcwxC09rn3gRlNtkSlzZK5_Zm4M4siloT-_dwv-063sAWgsxNPO__R5_a57aYIQKdcfeQkQPbfHgTTbRVSbB1nnskL5kA9Og143Vuhg', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -20.2402172, 'lng': -70.14509579999999}, 'viewport': {'northeast': {'lat': -20.23872782010728, 'lng': -70.14418122010727}, 'southwest': {'lat': -20.24142747989272, 'lng': -70.1468808798927}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Holiday Inn Express Iquique', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1367, 'html_attributions': ['<a href="https

{'html_attributions': [], 'next_page_token': 'ATtYBwLiFJO2Y1F_hA07Adk-37c9KzZDkQUR9HHGRmRVSLsXi6dBoivWpHV9D4YWM5DpOqEvJ2uzTZ4qWxZEojO1FUL7DOGeekJ-AWSfY2Wm2lubanZ7W9IfX64yjfN_IWgxdv_GnhFrBJ3ChxT-JkvlbVZHnsrWS9wFFoZLM6hxhFBoCY5Dlig8pOulS6P_bXPe4bRi8aWlIybf7nrHH_z5RpOC64bayOxgnVqjYj8nkq3t2aeCUQpUG9R5SIsKlzss8KeTxhuZbhJ0pWT9V5dVDHDSGNAyEqqKTKDcS3gpl4NBMx7jEwFA902AKocA0HwzOE2hC-7ha5PETkEXg_Yd524kVrP0UhzUkGD8wBIJWgIeNqPB6Y4YzBkhR3xDO2svKCjCamlGYy4AGlvofw5MvOgMT3FcdBjIvDKn3SYV2Un-HU7T_IJ9OMHDAJtBU8Q', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 21.871049, 'lng': 112.00871}, 'viewport': {'northeast': {'lat': 21.87244742989272, 'lng': 112.0100505798927}, 'southwest': {'lat': 21.86974777010728, 'lng': 112.0073509201073}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Yangjiang Country Garden Phoenix Hotel （West Gate）', 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/cont

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.3717616, 'lng': -50.8449652}, 'viewport': {'northeast': {'lat': -23.37036562010728, 'lng': -50.84352362010728}, 'southwest': {'lat': -23.37306527989272, 'lng': -50.84622327989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Sol Nascente', 'photos': [{'height': 4128, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116447899331908182108">Wanderley Anselmo Rocha</a>'], 'photo_reference': 'ATtYBwJymGVJp1OsNwuGtIsVOg6fiCCkji7pu_PCjJMg8QxqNJyHAtr6GpLxtuO9uQlkAo40CeI5kM6kKV-ri0y09HEIAfd_AqIoR5xpPxMA4Utwlp4k69g4pc81esM_-tH0mpS_vvk6ZCQnENG1_lN5MO6LOWxBPc9WIHjRq6VIP59S9BsE', 'width': 3096}], 'place_id': 'ChIJAxltXv4Z65QRyQZXT0obQ9U', 'plus_code': {'compound_code': 'J5H4+72 Assaí, State of Paraná, Brazil', 'global_code': '588FJ5H4+72'}, 'rating': 3.9, 'reference': 'ChIJAxltXv4Z65QRyQZXT0obQ9U', 'scope': 'GOOG

{'html_attributions': [], 'next_page_token': 'ATtYBwIwiEK5R54OBDS5QiYxAjLho6GJu8i6GdPPMY4u4x3wYQ5kVB-EAQ6opcLQ5a6x-PeocpjvEkjPq7Pop_YpUFN-_-omd9SfW36QgPE7_XECECgdC30UsKDI75vpm4ghfKr0Ks3YrpBLRzDOm-O-SgxIsZLtzEDEJmayKRNWrMysWzHWlzEdKhji1JUxaz9P8FlIit3597iaE4tlGgoBpGHRrIL-8H0uIuJwsHSEqScCR6I5AvLKKdJtnqN8GcPOCf2GQvDqqCCm0FBODVZVxH7GTMcK124rBiLt0X6OQvktptCTja6wqJga1FIIM_wsdNGgJC683SMFxb_df7Exizq54l6SHQq4mTtzX2PKy3_jN-TBKPsIYKsMQIh1eaymN5Y6CqnX7eikUsolhlxpauBjfqAGBMSmEXGbDyVlCdLSJzUfLgWUqT9mN_23rWE', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -26.1961926, 'lng': -58.2036268}, 'viewport': {'northeast': {'lat': -26.19422915, 'lng': -58.20228697010727}, 'southwest': {'lat': -26.19814095, 'lng': -58.20498662989271}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Howard Johnson Hotel And Casino Formosa', 'opening_hours': {'open_now': True}, 'photos': [{'height': 257, 'html_attributions': ['<a href="https://maps

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -5.173939499999999, 'lng': -40.667476}, 'viewport': {'northeast': {'lat': -5.172685470107278, 'lng': -40.66618867010729}, 'southwest': {'lat': -5.175385129892722, 'lng': -40.66888832989273}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'HOTEL POTY', 'photos': [{'height': 774, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107343303280694724272">Danielle Borges De Mesquita</a>'], 'photo_reference': 'ATtYBwJp7S5Eg34-1LkT0TV1gminDNxJoZQ6QuQnMjovYWD-ci3Uo2nQgSmJVPQZLWvs7BGo7x49QzCooZ60Q9dtpZX01TtOAHFJ3FA87xTSOHx-tdSIXjfgMSAsKsqX-N_13xpYT7I6Q3VUcGocdbbDzlSgXhi6u9q6nSRvd8pugJZI0h8', 'width': 1032}], 'place_id': 'ChIJj7CVkeH1lgcR46tyeBTst_o', 'plus_code': {'compound_code': 'R8GM+C2 Centro, Crateús - State of Ceará, Brazil', 'global_code': '686XR8GM+C2'}, 'rating': 4.5, 'reference': 'ChIJj7CVkeH1lgcR46tyeBTst_o', 'sco

...

{'html_attributions': [], 'next_page_token': 'ATtYBwLdYg3rhdOsaoG7hE2up_G5SCPJuxYxdZVq6F74l62kx8VPgaarJ_OgixPnoErnvJrNervkOgJ1d6qbVHCv30CXmOe5tiNDAA24dggcsP7E3diy-gXmAdo5t8Jjf3D5w9DsXDK_KwRZwvk_divhJRDC2UkjxhC5a4X0BLFSiaVE64cWJ8-LhCiezPoWGyqEw1OPoDWKrH-APWJ23AjRpMVve77ZfyAMgmcNJ_KIiC8yb-LkWQ74SZX1IgofyCuzx7WEbcTbloCA3cIEBAP16lrg50vonaDYx3pbCtoZVImHjpI-oEPAzfA2X4Won0_uS7kOgB5PRA0FV5oS8d0oMrNXTsZoaNbUGwjfEvyUGGqSKhh2xLE6MTbDjhYlR46cZMLfIyIDWhvu79OlNBvVuCgD0Xhp0ixXkB6BabESaBu-VCzFoh80YkAEPtZmCh8', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -45.4162124, 'lng': 167.7133478}, 'viewport': {'northeast': {'lat': -45.41494382010728, 'lng': 167.7148239298927}, 'southwest': {'lat': -45.41764347989272, 'lng': 167.7121242701073}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Distinction Luxmore Hotel Te Anau', 'opening_hours': {'open_now': True}, 'photos': [{'height': 800, 'html_attributions': ['<a href="https://

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -24.973516, 'lng': -55.1845906}, 'viewport': {'northeast': {'lat': -24.97253834999999, 'lng': -55.18363622010728}, 'southwest': {'lat': -24.97644895000001, 'lng': -55.18633587989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png', 'name': 'Cargill - Unidad ITAKYRY', 'photos': [{'height': 3096, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101008553961747528213">A Google User</a>'], 'photo_reference': 'ATtYBwJgVdSJfy6rE2g1fL8YCAuThYPA1pRSVAdST_9QunMNCdMHB5BKFs4mvtNattxqs5IKLWGHGKdqnIKws6aOdNKbzea9ZLb_BgEt5YqGAa_VbAF3j3UQKM1PzGnWC2XTll1Htnt8A1kxd86biK3PfZcgAAs4TrhV5v3wheThfdFORiUQ', 'width': 4128}], 'place_id': 'ChIJuzOcqq2Z9ZQR5NP9rIaW1yA', 'plus_code': {'compound_code': '2RG8+H5 Ykua Pora, Paraguay', 'global_code': '58762RG8+H5'}, 'rating': 3.6, 'reference': 'ChIJuzOcqq2Z9ZQR5NP9rIaW1yA', 'scope': 'GOOGLE', 't

{'html_attributions': [], 'next_page_token': 'ATtYBwKJQ1l1iMQRjfsmQ6NcGa_BKob3Gsx9IDOZH25WP3Rq0PJGtXWKQiKlXECl31S6I6gjeO-s-T61pPLGk7vjaI488_fiS6f77zmdAyUzP2Y6roaGIbh77_Jzx_8MX8euCGQLjNtNI4iDD8KtT0VafLJD0ZKsaKID-0MJZbOypN5Am7IdI-KYr_2WXjrIbsmmGi_hWKZm4D6hJNcrIUa2pyuqbEG6racSgHWUdVyQQBiV3_6dZmyy-i66bFwpnTYc1vseMtGCcB8adqb6oOfhU7O3lqfGSg3uXjpHNtcISHCx0xup2HxuIGEi_7EIJvzB7yhhVkpvFK4CWFb3M6nZR4Sz1PKl0p3Cw9pJdBeDcBKJr7xcDzNgXdZbPEPb6BqyOtJMam7TzIgBbA-Xm1a4309n2vy74cJTDoViRX5YIkDC6sYJ9WPTYR2k2AhjKT4', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -20.2372341, 'lng': 57.8005028}, 'viewport': {'northeast': {'lat': -20.23574782010728, 'lng': 57.80178617989272}, 'southwest': {'lat': -20.23844747989272, 'lng': 57.79908652010727}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Tropical Attitude Hotel', 'photos': [{'height': 3265, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113999049945124680

{'html_attributions': [], 'next_page_token': 'ATtYBwIUaH7wUB5JSFJsXfFHLoEdEk9-9Ud-rZ_s5Y10pNyUboHcLNdzPJVth0KdFOxGX8ADPLT_ZSec0MU7oQmhMp6KLHwrDskmzOe_KMYMPF_vfhDqLFesjbmsapw_U3Ej4DE0O17VN5n9hdCRA201mvoKtKwQVvp1-iWeWsKoKAZziPn5KvJmX_eg6HbwO3mEh4Ipem0ZQOYz_Sy8IqlUKjtQVmyqiDrU-NcMlLuIwM2DEOwG7cqPLEiNZG76ZUNn3P1DDrMsXodKmsXpEll9a1smSfu3K6YqwumtpX0KeTohSKjVX5VG57GRkm0i4b2xmFE3Vs8HPT-YBL9WN9LexXitVqVwijxY1Niu1AUw3Tg1Fn_yGid9ZFSpUPIkPIVMl7BzUwDcQDTuumTbu0YRJf7AQKutLKehUc-5Ve0RJ5zgQZNXkV25Rcnf31St-xY', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -17.7612204, 'lng': -63.2023465}, 'viewport': {'northeast': {'lat': -17.75984767010728, 'lng': -63.20107782010728}, 'southwest': {'lat': -17.76254732989272, 'lng': -63.20377747989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Marriott Santa Cruz de la Sierra Hotel', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1366, 'html_attributions': ['<a href="

{'html_attributions': [], 'next_page_token': 'ATtYBwIjhV3YpZSRMLECi3dm5PadPRLPeQ7rZjra5-3iWMgQXciE3dMaprov1jpVioKJINIInblv2F-EnvENJFx3FyB9_2N_UsS_ktjWdVay_o-frwjxH7xnoKxuQg9EzjgY3kufzMaAPdI0yTLcH_DO-HvBNHYhmZQA08RX8wJCqtvlYcFbEpNe9oq8XynNi04vzDcsY8INzT_arcA_9uaFnH99xC54RQiKMtE5hgCYClZGDjSosaz4VlVPZrtkgdgEtTSKRBmy4flnqWolVvxkvTdlbGVnvOsSKgksskZCQ-NjwBnCU8CdOcB1KcW-sX3cWlxvGuQN4oCK_BY_g1Ea4d2_n7WjM4t_Ug31O0h0CZnJuStJgIyZ0Qp_NGASuuEiNAzb1iC7avkT5--XOsMU-bE_PwhL_dvWrOtWXSU_bU6hXyeL3wRKYh7cahM9ILM', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -3.748993, 'lng': -73.24410759999999}, 'viewport': {'northeast': {'lat': -3.747684020107278, 'lng': -73.24279697010728}, 'southwest': {'lat': -3.750383679892722, 'lng': -73.24549662989273}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'DoubleTree by Hilton Iquitos', 'opening_hours': {'open_now': True}, 'photos': [{'height': 4160, 'html_attributions': ['<a href="https

In [14]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
125,Iquique,-20.2208,-70.1431,73.40,49,0,9.22,CL,1612661958,Holiday Inn Express Iquique
135,Yangjiang,21.8500,111.9667,78.01,45,0,3.49,CN,1612661972,Yangjiang Country Garden Phoenix Hotel （West G...
152,Assaí,-23.3733,-50.8414,75.20,57,0,5.75,BR,1612661996,Hotel Sol Nascente
170,Vao,-22.6667,167.4833,78.03,80,0,5.46,NC,1612662023,OURE TERA beach resort
174,Lázaro Cárdenas,17.9583,-102.2000,73.83,75,0,1.74,MX,1612662029,Hotel de Casablanca
176,Formosa,-26.1775,-58.1781,71.60,78,0,5.75,AR,1612662032,Howard Johnson Hotel And Casino Formosa
249,Crateús,-5.1783,-40.6775,76.32,57,0,3.04,BR,1612662136,HOTEL POTY
274,Puerto Baquerizo Moreno,-0.9000,-89.6000,76.73,85,0,7.78,EC,1612662138,Casa Blanca
278,Ban Phai,16.0600,102.7310,78.01,50,0,7.40,TH,1612662177,Jasper Hotel Ban Phai โรงแรมแจสเปอร์โฮเทล
283,Te Anau,-45.4167,167.7167,75.90,38,0,5.68,NZ,1612662184,Lakefront Lodge Te Anau


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations)
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))